# Dataset analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import utils

In [2]:
features = [
    'sx_x',
    'sx_y',
    'sx_z',
    'sx_roll',
    'sx_pitch',
    'sx_yaw',
    'sx_thumb',
    'sx_forefinger',
    'sx_middle_finger',
    'sx_ring_finger',
    'sx_little_finger',
    'dx_x',
    'dx_y',
    'dx_z',
    'dx_roll',
    'dx_pitch',
    'dx_yaw',
    'dx_thumb',
    'dx_forefinger',
    'dx_middle_finger',
    'dx_ring_finger',
    'dx_little_finger'
    ]

In [3]:
path = "tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))
print("thank-1.tsd")
s = 0
for filename in filenames:
    s += len(pd.read_csv(path+weeks[1]+"/"+filename, names=features, sep='\t',))
print("Average duration of each time-series for week 1:",s/len(filenames))
df = pd.read_csv(path+weeks[1]+"/"+"thank-1.tsd", names=features, sep='\t',)
df.head(11)[['dx_x', 'sx_x']]

thank-1.tsd
Average duration of each time-series for week 1: 54.463157894736845


,dx_x,sx_x
0,-0.065836,-0.045284
1,-0.083517,-0.042292
2,-0.083874,-0.041399
3,-0.092269,-0.041801
4,-0.108700,-0.040997
5,-0.070613,-0.039211
6,-0.062755,-0.039970
7,-0.073114,-0.040640
8,-0.072310,-0.039256
9,-0.072533,-0.038943


In [4]:
for i in df.columns:
    print(f"{i}:,\t{np.min(df[i]):.3f}, \t{np.mean(df[i]):.3f}, \t{np.max(df[i]):.3f}")
    #print(f"{i}:,\t{np.mean(df[i]):.3f}, \t{np.std(df[i]):.3f}")

sx_x:,	-0.045, 	-0.040, 	-0.039
sx_y:,	-0.080, 	-0.078, 	-0.076
sx_z:,	-0.072, 	-0.070, 	-0.068
sx_roll:,	0.475, 	0.478, 	0.481
sx_pitch:,	0.346, 	0.348, 	0.350
sx_yaw:,	0.557, 	0.559, 	0.560
sx_thumb:,	0.000, 	0.167, 	0.241
sx_forefinger:,	0.000, 	0.000, 	0.000
sx_middle_finger:,	0.000, 	0.000, 	0.000
sx_ring_finger:,	0.000, 	0.000, 	0.000
sx_little_finger:,	0.052, 	0.052, 	0.057
dx_x:,	-0.109, 	0.011, 	0.065
dx_y:,	-0.079, 	-0.003, 	0.270
dx_z:,	-0.081, 	-0.046, 	0.127
dx_roll:,	0.063, 	0.346, 	0.456
dx_pitch:,	0.356, 	0.472, 	0.834
dx_yaw:,	0.476, 	0.530, 	0.770
dx_thumb:,	0.000, 	0.069, 	0.141
dx_forefinger:,	0.000, 	0.000, 	0.000
dx_middle_finger:,	0.000, 	0.002, 	0.032
dx_ring_finger:,	0.000, 	0.000, 	0.000
dx_little_finger:,	0.000, 	0.033, 	0.088


In [5]:
weeks

['tctodd1',
 'tctodd2',
 'tctodd3',
 'tctodd4',
 'tctodd5',
 'tctodd6',
 'tctodd7',
 'tctodd8',
 'tctodd9']

In [6]:
features_1 = [
    'sx_roll',
    'sx_pitch',
    'sx_yaw',
    'sx_thumb',
    'sx_x',
    'sx_y',
    'sx_z',
    'sx_forefinger',
    'sx_middle_finger',
    'sx_ring_finger',
    'sx_little_finger',
    'dx_roll',
    'dx_pitch',
    'dx_yaw',
    'dx_x',
    'dx_y',
    'dx_z',
    'dx_thumb',
    'dx_forefinger',
    'dx_middle_finger',
    'dx_ring_finger',
    'dx_little_finger'
    ]

df_inv = pd.read_csv(path+weeks[1]+"/"+"thank-1.tsd", names=features_1, sep='\t',)
df_inv.iloc[30:46][['sx_x', 'dx_x', 'sx_y', 'dx_y', 'sx_z', 'dx_z']]

,sx_x,dx_x,sx_y,dx_y,sx_z,dx_z
30,0.348067,0.446654,0.558666,0.360518,0.161137,0.480483
31,0.349239,0.446190,0.558812,0.360958,0.161137,0.480996
32,0.349092,0.445860,0.558324,0.361446,0.161137,0.480947
33,0.348897,0.445238,0.558153,0.361519,0.161137,0.481118
34,0.349068,0.444688,0.558568,0.361055,0.161137,0.481607
35,0.349361,0.444224,0.558702,0.360933,0.161137,0.482156
36,0.348775,0.444017,0.558409,0.360518,0.161137,0.482168
37,0.347993,0.443822,0.558678,0.359102,0.161137,0.482144
38,0.348091,0.443443,0.559069,0.358223,0.161137,0.482778
39,0.348457,0.443016,0.559142,0.357613,0.161137,0.483364


In [7]:
for i in df_inv.columns:
    #print(f"{i}:,\t{np.min(df_inv[i]):.3f}, \t{np.max(df_inv[i]):.3f}")
    print(f"{i}:,\t{np.mean(df_inv[i]):.3f}, \t{np.std(df_inv[i]):.3f}")

sx_roll:,	-0.040, 	0.001
sx_pitch:,	-0.078, 	0.001
sx_yaw:,	-0.070, 	0.001
sx_thumb:,	0.478, 	0.001
sx_x:,	0.348, 	0.001
sx_y:,	0.559, 	0.000
sx_z:,	0.167, 	0.075
sx_forefinger:,	0.000, 	0.000
sx_middle_finger:,	0.000, 	0.000
sx_ring_finger:,	0.000, 	0.000
sx_little_finger:,	0.052, 	0.001
dx_roll:,	0.011, 	0.053
dx_pitch:,	-0.003, 	0.104
dx_yaw:,	-0.046, 	0.043
dx_x:,	0.346, 	0.153
dx_y:,	0.472, 	0.163
dx_z:,	0.530, 	0.078
dx_thumb:,	0.069, 	0.061
dx_forefinger:,	0.000, 	0.000
dx_middle_finger:,	0.002, 	0.007
dx_ring_finger:,	0.000, 	0.000
dx_little_finger:,	0.033, 	0.028


In [14]:
path = "tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

Had to change her label in tctodd1 in his_hers. Otherwise 96 labels instead of 95.
vedere se le varie label di his_hers sono statisticamente simili (che vuol dire statisticamente? boh guardiamo media e dev. std)

In [15]:
df = pd.DataFrame(data, columns=['label', 'time_series'])
df['label'].value_counts()
df.head()

,label,time_series
0,0,"[[-0.064909, 0.034318, -0.043964, 0.626383, 0...."
1,0,"[[-0.107059, -0.126109, -0.053742, 0.612516, 0..."
2,0,"[[-0.061427, -0.082576, -0.102991, 0.735469, 0..."
3,1,"[[-0.128178, 0.02695, -0.050126, 0.455028, 0.4..."
4,1,"[[-0.143672, -0.144416, -0.047447, 0.660979, 0..."


In [50]:
df.to_csv("dataset.csv", index=False)

In [17]:
np.mean(df['time_series'].iloc[0], axis=0)
np.max(df['time_series'].iloc[0], axis=0)

array([ 0.019613,  0.090933,  0.007249,  0.649138,  0.636772,  0.513242,
        1.      ,  1.      ,  1.      ,  1.      ,  1.      ,  0.113879,
        0.46191 , -0.004575,  0.432945,  0.839049,  0.666005,  1.      ,
        1.      ,  1.      ,  1.      ,  1.      ])

In [19]:
all_stacked = np.vstack(df['time_series'])
np.max(all_stacked, axis=0)

array([0.284053, 0.45193 , 0.27958 , 0.997842, 0.993448, 0.674636,
       1.      , 1.      , 1.      , 1.      , 1.      , 0.505725,
       0.554828, 0.292127, 0.997684, 0.997696, 0.987548, 1.      ,
       1.      , 1.      , 1.      , 1.      ])

In [44]:
df.iloc[0]['time_series'].shape

(90, 22)

In [20]:
import sklearn as sk
from sklearn.model_selection import train_test_split
seed = 0

# Shuffle the dataframe
shuffled_df = sk.utils.shuffle(df, random_state=seed)
X_train, X_test, y_train, y_test = train_test_split(shuffled_df['time_series'], shuffled_df['label'], stratify=shuffled_df['label'],  train_size=.7, random_state=seed)

In [46]:
# each row of the Series object is an array. Classifiers won't read it. We create a matrix of values.
def from_series_to_matrix(num_predictors:int, time_series:pd.Series) -> np.ndarray:
    """Function used to transform the pandas Series to a matrix.
    Used to feed classifiers.

    Args:
        num_predictors (int): numbers of predictors
        time_series (pd.Series): time series data

    Returns:
        np.ndarray: NxM matrix where:
        - N is the number of examples
        - M is the number of predictors
    """
    a = np.zeros(shape=(len(time_series), num_predictors))
    for i in range(len(time_series)):
        for j in range(num_predictors):
            a[i,j] = time_series.iloc[i][j]
    return a

# compute_mean_feature_vector used to compute baseline
def compute_mean_feature_vector(time_series:pd.Series) -> np.ndarray:
    """Compute the mean of each field of each time series example

    Args:
        time_series (pd.Series): time series

    Returns:
        np.ndarray: feature vector for each entry
    """
    num_predictors = time_series.iloc[0].shape[-1]
    return from_series_to_matrix(num_predictors, time_series.apply(lambda x: np.mean(x, axis=0)))

X_train_mean_fv = compute_mean_feature_vector(X_train)
X_test_mean_fv = compute_mean_feature_vector(X_test)

In [47]:
X_train_mean_fv.shape

(1795, 22)

In [48]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(X_train_mean_fv)
matrix_X_train_mean_fv_scaled = SC.transform(X_train_mean_fv)
matrix_X_test_mean_fv_scaled = SC.transform(X_test_mean_fv)

svc = SVC()
svc.fit(matrix_X_train_mean_fv_scaled, y_train)

predictions = svc.predict(matrix_X_test_mean_fv_scaled)

print(accuracy_score(y_test, predictions))

0.7519480519480519


-------------

In [2]:
df = pd.read_json("data_no_bool.json").T
df.head()

,label,time_series
0,6,"[{'BallAcceleration': 1636.7987723122642, 'Dis..."
1,-1,"[{'BallAcceleration': 0.0, 'DistanceWall': 39...."
2,-1,"[{'BallAcceleration': 124246.29375405672, 'Dis..."
3,-1,"[{'BallAcceleration': -8210.634011562943, 'Dis..."
4,-1,"[{'BallAcceleration': 1197.5360615055833, 'Dis..."


In [3]:
df.label.value_counts()

 2    60
 6    46
 5    45
-1    43
 3    38
 7    36
 1    30
Name: label, dtype: int64

----------

In [4]:
#from sklearn.utils import shuffle
import sklearn as sk
from sklearn.model_selection import train_test_split
seed = 0

# Shuffle the dataframe
shuffled_df = sk.utils.shuffle(df, random_state=seed)
shuffled_df.head()

,label,time_series
111,6,"[{'BallAcceleration': 4841.49275538644, 'Dista..."
175,6,"[{'BallAcceleration': -347.5747892882355, 'Dis..."
12,7,"[{'BallAcceleration': 0.0, 'DistanceWall': 400..."
218,1,"[{'BallAcceleration': 106808.88211978199, 'Dis..."
205,7,"[{'BallAcceleration': 14189.044826174126, 'Dis..."


In [5]:
from utils import extract_matrix
# Create the matrix version
shuffled_df.time_series = shuffled_df.time_series.apply(extract_matrix)
shuffled_df["shape"] = shuffled_df.time_series.apply(lambda x: x.shape)
shuffled_df.head()

,label,time_series,shape
111,6,"[[4841.49275538644, 3920.09, 2013.1, 164.34092...","(19, 17)"
175,6,"[[-347.5747892882355, 4002.97, 2013.0, 218.624...","(19, 17)"
12,7,"[[0.0, 4004.29, 2013.08, 193.23514638905624, 1...","(16, 17)"
218,1,"[[106808.88211978199, 13.230000000000018, 345....","(19, 17)"
205,7,"[[14189.044826174126, 3329.59, 2013.15, 153.21...","(16, 17)"


In [6]:
X_train, X_test, y_train, y_test = train_test_split(shuffled_df['time_series'], shuffled_df['label'], train_size=.75, random_state=seed)

In [7]:
len(X_train)

223

In [8]:
len(X_test)

75

In [9]:
a = y_train.value_counts()
a/=np.sum(a)
a

 2    0.219731
 5    0.161435
 6    0.161435
-1    0.134529
 3    0.125561
 7    0.103139
 1    0.094170
Name: label, dtype: float64

In [10]:
a = y_test.value_counts()
a/=np.sum(a)
a

 7    0.173333
-1    0.173333
 2    0.146667
 3    0.133333
 6    0.133333
 5    0.120000
 1    0.120000
Name: label, dtype: float64

# Baseline
Use as feature vector the mean of each field of the time series

In [11]:
X_train_mean_fv = utils.compute_mean_feature_vector(X_train)
X_test_mean_fv = utils.compute_mean_feature_vector(X_test)

In [12]:
matrix_X_train_mean_fv = utils.from_series_to_matrix(17, X_train_mean_fv)
matrix_X_test_mean_fv = utils.from_series_to_matrix(17, X_test_mean_fv)

In [13]:
prova_train = y_train.to_numpy().astype('int')
prova_test = y_test.to_numpy().astype('int')

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(matrix_X_train_mean_fv)
matrix_X_train_mean_fv_scaled = SC.transform(matrix_X_train_mean_fv)
matrix_X_test_mean_fv_scaled = SC.transform(matrix_X_test_mean_fv)

svc = SVC()
svc.fit(matrix_X_train_mean_fv_scaled, prova_train)

predictions = svc.predict(matrix_X_test_mean_fv_scaled)

print(accuracy_score(prova_test, predictions))

0.7866666666666666
